You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:

# from utils import get_openai_api_key
# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# from langchain.llms import Ollama
# llm = Ollama(base_url="http://localhost:11434/v1", model="mistral:latest")

import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "sk-proj-1111" # dummy key
llm = ChatOpenAI(
    model="ollama/mistral:latest",
    base_url="http://localhost:11434/v1"
)

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    # verbose=2
    verbose=True,
    llm=llm
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Title: Navigating the Future of Artificial Intelligence: Trends, Players, and Opportunities for Your Business

Introduction:
Welcome to our comprehensive guide on Artificial Intelligence (AI). In this article, we'll delve into the latest trends shaping AI, explore the key players making waves in the industry, and discuss the potential impact of AI on your business. With an ever-evolving landscape, staying informed about AI is crucial for making educated decisions.

Target Audience:
Our primary audience consists of business owners, entrepreneurs, and professionals interested in leveraging A

I now can give a great answer
Final Answer:
# The Latest Trends and Key Players in Artificial Intelligence

## Introduction

Artificial Intelligence (AI) has become an integral part of our daily lives, revolutionizing industries and changing the way we interact with technology. As AI continues to advance at a rapid pace, it is crucial to stay updated on the latest trends and key players shaping the future of this field. In this blog post, we will delve into the latest trends in AI, introduce the key players driving innovation, and explore noteworthy news in the realm of Artificial Intelligence.

## Latest Trends in Artificial Intelligence

### Machine Learning Advancements
One of the most significant trends in AI is the continuous advancements in machine learning algorithms. Machine learning has enabled computers to learn from data and make predictions or decisions without being explicitly programmed, leading to breakthroughs in various applications such as image recognition, natural l

In [12]:
type(result)

crewai.crews.crew_output.CrewOutput

In [13]:
# print(result)

```markdown
Title: Navigating the Future of Artificial Intelligence: Trends, Players, and Opportunities for Your Business

Introduction:
Welcome to the future as we guide you through the ever-evolving landscape of Artificial Intelligence (AI). In this article, we'll delve into the latest trends shaping AI, highlight key players making waves in the industry, and showcase real-world applications across various sectors. By understanding AI's potential impact on your business, you can make informed decisions to stay ahead of the curve.

**The State of Artificial Intelligence:**
- **Current trends and advancements in AI:** The ongoing development of machine learning, deep learning, and natural language processing has transformed AI capabilities. (SEO: AI Trends 2023)
- **Key players shaping the AI landscape:** From industry titans like Google and Microsoft to promising startups, these companies are redefining the future of AI. (SEO: Key Players in the AI Industry)
- **Emerging technologies:

- Display the results of your execution as markdown in the notebook.

In [14]:
from IPython.display import Markdown
# Markdown(result)
Markdown(str(result))

```markdown
Title: Navigating the Future of Artificial Intelligence: Trends, Players, and Opportunities for Your Business

Introduction:
Welcome to the future as we guide you through the ever-evolving landscape of Artificial Intelligence (AI). In this article, we'll delve into the latest trends shaping AI, highlight key players making waves in the industry, and showcase real-world applications across various sectors. By understanding AI's potential impact on your business, you can make informed decisions to stay ahead of the curve.

**The State of Artificial Intelligence:**
- **Current trends and advancements in AI:** The ongoing development of machine learning, deep learning, and natural language processing has transformed AI capabilities. (SEO: AI Trends 2023)
- **Key players shaping the AI landscape:** From industry titans like Google and Microsoft to promising startups, these companies are redefining the future of AI. (SEO: Key Players in the AI Industry)
- **Emerging technologies:** Discover how breakthroughs such as edge computing, quantum computing, and generative models are pushing the boundaries of AI.

**Real-World Applications of AI:**
- **Case studies:** Unveil how AI is revolutionizing industries like healthcare, finance, retail, and more. (SEO: Real-world applications of AI)
- **Addressing common pain points:** Learn about the ways AI can streamline operations, enhance customer service, and foster growth for your business.
- **Future potential:** Gain insights into the limitless possibilities of AI and its impact on various sectors in the coming years.

**Navigating the AI Market:**
- **Identifying suitable solutions:** Tips for finding the ideal AI solution to cater to your unique needs.
- **Popular AI platforms and tools:** Overview of the most popular platforms, tools, and services available for businesses.
- **Partnering with AI service providers:** Factors to consider when collaborating with AI service providers or integrating AI technologies.

**Making Informed Decisions About AI Adoption:**
- **Data privacy, ethics, and transparency:** The paramount importance of prioritizing these values during your AI adoption journey. (SEO: Data Privacy, Ethics, and Transparency in AI Adoption)
- **Measuring the ROI of AI investments:** Strategies for evaluating the return on investment of your AI initiatives. (SEO: Measuring the ROI of AI Investments)
- **Best practices for adopting AI within your organization:** Tips for a smooth and successful AI integration process.

**Call to Action:**
- **Explore AI solutions:** Encourage readers to find AI solutions tailored to their specific needs. (SEO: AI in [Industry Name]: Case Studies and Best Practices)
- **Access resources:** Offer a variety of resources, such as whitepapers, webinars, and case studies, to support informed AI decision-making. (SEO: A Guide to Evaluating AI Solutions for Your Business)
- **Connect with us:** Invite readers to share their experiences with AI or ask questions for future articles. (SEO: Data Privacy, Ethics, and Transparency in AI Adoption)
```

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "Transformer Model: Attention is all you need"
result = crew.kickoff(inputs={"topic": topic})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Transformer Model: Attention is all you need.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Content Plan for "Transformer Model: Attention is all you need"

Target Audience: Data scientists, machine learning enthusiasts, and software developers with some knowledge of deep learning and neural networks. Interests include natural language processing (NLP), artificial intelligence (AI), and computational linguistics. Pain points may include difficulty in understanding complex models and their applications, and a need for practical examples and solutions.

Content Outline:

1. Introduction
   - Briefly introduce the Transformer model and its importance in the fiel

I now can give a great answer

Final Answer:
# Understanding the Transformer Model: Attention is all you need

## Introduction
In the realm of natural language processing, the Transformer Model has emerged as a game-changer, revolutionizing the way we approach deep learning tasks. At the heart of this innovative model lies the attention mechanism, a powerful concept that has reshaped the landscape of NLP. Developed by Vaswani et al. in 2017, the Transformer architecture has gained widespread popularity for its ability to handle sequential data with unparalleled efficiency and effectiveness.

## Latest Trends
The latest advancements in Transformer-based models, such as BERT, GPT-3, and T5, have pushed the boundaries of what is possible in NLP. These models have demonstrated remarkable performance on a variety of tasks, from language understanding to text generation. Companies and research institutions like Google, OpenAI, and Facebook AI Research have been at the forefront of developing

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm1 = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm2 = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).